In [4]:
import os
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from newsdataapi import NewsDataApiClient

# API key authorization, Initialize the client with your API key

#api_newsdata = NewsDataApiClient(apikey="YOUR_API_KEY")

# You can pass empty or with request parameters {ex. (country = "us")}

#response = api_newsdata.news_api( q= "ronaldo" , country = "us")

In [2]:
# Load .env enviroment variables
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API"])
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_2"])
#api_newsdata = NewsDataApiClient(apikey="newsdata")

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [124]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword,fdate,edate):
    all_headlines = []
    all_contents = []
    all_dates = [] 
    # string conversion of dates passed in to function
    fdate_str=str(fdate.strftime("%Y-%m-%d"))
    edate_str=str(edate.strftime("%Y-%m-%d"))

    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    #while date > end_date:
    print(f"retrieving news from: {fdate} - {edate}")
    articles = newsapi.get_everything(
        q=keyword,
        # from_param=str(date)[:10],
        # to=str(date)[:10],
        from_param=fdate_str,
        to=edate_str,
        language="en",
        sort_by="relevancy",
        page=1,
    )
    headlines = []
    contents = []
    for i in range(0, len(articles["articles"])):
        headlines.append(articles["articles"][i]["title"])
        contents.append(articles["articles"][i]["content"])
    all_headlines.append(headlines)
    all_contents.append(contents)
    all_dates.append(fdate)
    #date = date - timedelta(weeks=1)
    return all_headlines, all_dates, all_contents

# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(data,sdate):
    df=data.copy()
    sentiment = []
    sentiment_pos = []
    sentiment_neg = []
    for day in data:
        day_score = []
        day_positive = []
        day_negative = []
        for h in day:
            
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
                day_positive.append(sid.polarity_scores(h)["pos"])
                day_negative.append(sid.polarity_scores(h)["neg"])
        sentiment.append(sum(day_score) / len(day_score))
        sentiment_pos.append(sum(day_positive) / len(day_positive))
        sentiment_neg.append(sum(day_negative) / len(day_negative))
    d={"c0":sentiment,"p0":sentiment_pos,"n0":sentiment_neg,"Date":str(sdate.strftime("%Y-%m-%d"))}
    sentiment_df=pd.DataFrame(data=d).set_index("Date")
    return sentiment_df #, sentiment_pos, sentiment_neg

In [45]:
# Setting up Variable initial values
# current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
# past_date = pd.Timestamp(datetime.now()- timedelta(7), tz="America/New_York").isoformat()
# date = datetime.strptime(current_date[:10], "%Y-%m-%d")
# end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
fdate = datetime(2022, 6, 1)
edate = fdate + timedelta(days=1)
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [46]:
btc_headlines, btc_dates, btc_contents = get_headlines("bitcoin",fdate,edate)

# eth_headlines, eth_dates = get_headlines("ethereum")
# gold_headlines, gold_dates = get_headlines("gold")
# crude_oil_headlines, crude_oil_dates, crude_oil_contents = get_headlines("crude oil")
# sp500_headlines, sp500_dates = get_headlines("S&P 500")

Fetching news about 'bitcoin'
******************************
retrieving news from: 2022-05-26 00:00:00 - 2022-06-01 00:00:00


In [1]:
#btc_avg_compound_sentiment,btc_avg_pos_sentiment,btc_neg_sentiment = headline_sentiment_summarizer_avg(btc_contents)
#btc_avg_compound_sentiment = headline_sentiment_summarizer_avg(btc_contents)
btc_sentiment_df=headline_sentiment_summarizer_avg(btc_contents,fdate)
btc_sentiment_df
#btc_contents
type(btc_sentiment_df.index)

NameError: name 'headline_sentiment_summarizer_avg' is not defined

In [43]:
fdate = datetime(2022, 5, 25)
edate = fdate + timedelta(days=6)
print(fdate.strftime("%Y-%m-%d"))
print(edate.strftime("%Y-%m-%d"))
str(fdate.strftime("%Y-%m-%d"))

2022-05-25
2022-05-31


'2022-05-25'

In [79]:
df = pd.DataFrame(btc_avg_compound_sentiment,pd.DatetimeIndex(btc_dates).rename("Date"))
df.rename(columns={0:"Sentiment_BTC"},inplace=True)

In [80]:
df

,Sentiment_BTC
Date,
2022-05-26,0.098313


In [5]:
dt.datetime.now()

datetime.datetime(2022, 6, 26, 21, 1, 8, 181918)